## Data on potential risk factors:  Co-infections (determine whether co-existing respiratory/viral infections make the virus more transmissible or virulent) and other co-morbidities

In [122]:
import os
import json
from pprint import pprint
from copy import deepcopy

import numpy as np
import pandas as pd
import numpy as np
import json
import glob
import sys

sys.path.insert(0, "../")

root_path = '/Users/u6066091/Desktop/kaggle/input/corona_challenge/'

corona_features = {"doc_id": [None], "source": [None], "title": [None],
                  "abstract": [None], "text_body": [None]}
corona_df = pd.DataFrame.from_dict(corona_features)

json_filenames = glob.glob(f'{root_path}/**/*.json', recursive=True)

In [3]:

root = '/Users/u6066091/Desktop/kaggle'

In [6]:
root = '/Users/u6066091/Desktop/kaggle'
sub_que = pd.read_csv(root + '/input/kaggle_annotated/COVID-19_dictionary_sub_question.csv')
sub_que.head()

,Task,Question,Subquestion,Outcome.list,Differences.list,Design.list,Notes
0,"What is known about transmission, incubation, ...",Range of incubation periods for the disease in...,Range of incubation periods for humans: genera...,incubation period,age.,"contact tracing, survival analysis, time-to-ev...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...
1,"What is known about transmission, incubation, ...",Range of incubation periods for the disease in...,Range of incubation periods for humans: by age...,"incubation period, stratified by age group",age.,"contact tracing, survival analysis, time-to-ev...",Notes
2,"What is known about transmission, incubation, ...",Range of incubation periods for the disease in...,Range of incubation periods for humans: by hea...,"incubation period, stratified by health status...","health status (diabetes, hypertension, heart d...","contact tracing, survival analysis, time-to-ev...",Notes
3,"What is known about transmission, incubation, ...",Range of incubation periods for the disease in...,"How long are individuals contagious, even afte...","infectious period, serial interval, generation...","age, health status (diabetes, hypertension, he...",excretion profiles from experimental infection...,https://idpjournal.biomedcentral.com/articles/...
4,"What is known about transmission, incubation, ...",Prevalence of asymptomatic shedding and transm...,Prevalence of asymptomatic shedding: general p...,"nasal carriage, asymptomatic shedding","age, health status (diabetes, hypertension, he...",population prevalence survey using nasopharyng...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...


In [55]:
co_infec  = sub_que[sub_que['Question'] == 'Data on potential risk factors:  Co-infections (determine whether co-existing respiratory/viral infections make the virus more transmissible or virulent) and other co-morbidities']


In [103]:
co_infect = set(co_infec['Differences.list'])
design =  set(co_infec['Design.list'])
des_list = list(des_list)
co_infect = list(co_infect)
co_infect.extend(['progressive infiltrate','new infiltrate'])
co_infect

['HIV, AIDS',
 'new or progressive infiltrate on chest radiograph, leukocytosis, fever',
 'tuberculosis, tuberculosis stage, chest x-ray',
 'influenza',
 'progressive infiltrate',
 'new infiltrate']

In [105]:
list_des = list(set(des_list[0]+des_list[1]))
list_des.extend(['case-control','hospital infection control'])
list_des

[' matched case-control',
 ' medical records review',
 ' cross-sectional case-control',
 ' survival analysis',
 ' hospital-acquired infection surveillance',
 ' observational case series',
 ' logistic regression',
 ' mortality records review',
 'hospital infection control epidemiology',
 ' retrospective cohort',
 ' time series analysis',
 'risk factor analysis',
 ' outbreak investigation',
 'case-control',
 'hospital infection control']

In [72]:
import re
import json
co_infect_list = []
for i in range(0,len(json_filenames)):
    
    with open(json_filenames[i], "r") as infile:
        filetext = infile.read()
    with open(json_filenames[i]) as data:
        data = json.load(data)
        
        pattern = r'"text": "([^"]*(%s)[^"]*)((?!"section").)*"section": "([^"]*)"' % "|".join(co_infect)
        regex = re.compile(pattern, re.DOTALL)
   
        results = regex.findall(filetext)
        for match in results:
            co_infect_list.append([data['paper_id'], match[3], match[1], match[0]])
#             print ('Match in section "%s":\n %s \n-------------------' % (match[3], match[0]))

In [75]:
import pandas as pd
co_infect_sections = pd.DataFrame(co_infect_list, columns = ['doc_id', 'section', 'co_infect', 'text'])

In [109]:
sec_risk = []
for i in range(0, len(co_infect_sections['text'])):
    text = co_infect_sections.loc[i,'text']
    design = list_des
    design_list = [re.findall(ri.lower(), text) for ri in design]
    for des in design_list:
        if des != []:
            sec_risk.append([co_infect_sections.loc[i,'doc_id'],co_infect_sections.loc[i,'section'],des,text, co_infect_sections.loc[i,'co_infect']])
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   

In [112]:
co_infect = pd.DataFrame(sec_risk, columns = ['doc_id','section','design', 'text', 'co-infect'])
co_infect.to_csv('/Users/u6066091/Desktop/kaggle/output/dataframe/co_infect.csv')

In [120]:
cov_list = []
for i in range(0,len(co_infect)):
    covid = [re.findall('cov',co_infect.loc[i,'text'])]
    for cov in covid:
        if cov != []:
            cov_list.append(co_infect.loc[i,'text'])
